In [40]:
import torch, numpy as np,  pandas as pd, torch.nn as nn

In [57]:
df = pd.read_csv('iris.csv')
df['variety'].unique()

array(['Setosa', 'Versicolor', 'Virginica'], dtype=object)

In [85]:
torch.Tensor(df.loc[1][:-1].to_numpy().astype('float32'))

tensor([4.9000, 3.0000, 1.4000, 0.2000])

In [76]:
#One hot encoder - посмотреть
one_hoter = {'Iris-setoza': [1, 0, 0], 'Iris-virginica': [0, 1, 0], 'Iris-versocolor': [0, 0, 1]}

In [77]:
#В торче модель это отдельный класс, который должен наследовать nn.Module

class Net(nn.Module):
    #В инициализации находятся объекты, чьи состояния мы должны запомнить
    def __init__(self):
        super(Net, self).__init__()
        self.linear1 = nn.Linear(4, 32) #на вход 4 арг, на выход 32| 4 т.к. 4 параметра всего, 32 - подобрали эксперементально
        self.f_act = nn.ReLU() # фун. актив
        self.linear2 = nn.Linear(32, 3) #на выход 3 так как 3 класса всего
        self.softmax = nn.Softmax(dim=1)

    # В форварде описываем как мы будем эти штуки использовать в for
    def forward(self, x):
        res = self.linear1(x)
        res = self.f_act(res)
        res = self.linear2(res)
        res = self.softmax(res)
        return res

In [78]:
from torch.utils.data import Dataset, DataLoader
# Занимаются загрузкой данных и их вызовом
#датасет возвращает 1, dataloader множество

In [79]:
#Класс того как будет выдаваться один объект

class IrisDataset(Dataset):
    def __init__(self, dataframe: pd.DataFrame):
        self.df = dataframe

    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, ind):
        raw = self.df.loc[ind].to_numpy()
        target = raw[-1]
        target = torch.Tensor(one_hoter[target])
        samples = torch.Tensor(raw[:-1].astype('float32'))
        return samples, target

In [80]:
from torchsummary import summary
summary(Net())

TypeError: summary() missing 1 required positional argument: 'input_size'

In [81]:
#В керосе можно .fit, в торче нельзя

In [82]:
#функция обучающая одну эпоху
def train_one_epoch(epoch, trainloader, optimizer, model, loss, dataset):
    #кол-во эпох, оптимизатор - из старых весов отнимаем значения градиента и получаем новое
    correct = 0
    epoch_loss = 0.0 #для себя чтобы следить за изменение й ошибки
    for i, data in enumerate(trainloader, 0):
        x, y = data #так как датасет возвращает 2 значения
        optimizer.zero_grad()
        outputs = model(x) #результаты модели
        loss = loss(outputs, y) #функция ошибки
        loss.backward() #обратные градиенты (т.е считает призводную кажд элемента)
        epoch_loss += outputs.shape[0] * loss.item() #записываем значения ошибок
        optimizer.step() #совершает ход вычитания производных и старых w
        outputs = (outputs > 0.5).float() #для себя
        correct += (outputs == y).float().sum()

    print(f'TRAIN [{epoch + 1}] loss: {epoch_loss/ dataset.__len__():.3f}')


In [83]:
dataset = IrisDataset(df)
trainloader = DataLoader(dataset, batch_size=32, shuffle = True) #Всегда стоит шафлить
net = Net()
optimizer = torch.optim.RMSprop(net.parameters())
loss = nn.CrossEntropyLoss()

In [84]:
for epoch in range(10):
    train_one_epoch(epoch, trainloader, optimizer, net, loss, dataset)

KeyError: 'Setosa'